In [1]:
#from ucimlrepo import fetch_ucirepo
#
#data = fetch_ucirepo(id=76)
#pdf = data.data.features

import pandas as pd

pdf = pd.read_csv('data/train.csv')
pdf = pdf.drop(columns=['id', 'target'])

Mounted at /content/drive


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("K-Mode-global") \
    .getOrCreate()

sc = spark.sparkContext

In [3]:
df = spark.createDataFrame(pdf)
P = len(df.columns)
df.cache()
rdd = df.rdd.repartition(10)
rdd.cache()

MapPartitionsRDD[18] at coalesce at NativeMethodAccessorImpl.java:0

In [5]:
import numpy as np

In [6]:
def to_numpy(row):
  a = list(row.asDict().values())
  return np.array(a, dtype="<U22")

In [7]:
from math import e
import random
import numpy as np

K = 3 #number of cluster

for (idx, row) in enumerate(rdd.takeSample(withReplacement=False, num=K)):
  if idx == 0:
    centroid = np.array([to_numpy(row)]) #shape (1, P)
  else:
    c_ = to_numpy(row)
    c_ = np.array([c_]) #shape (1, P)
    centroid = np.concatenate([centroid, c_], axis=0)

centroid = sc.broadcast(centroid) #shape (K, P)
centroid.value

array([['0.0', '0.0', '0.0', 'F', 'Y', 'Blue', 'Circle', 'Hamster',
        'Finland', 'Oboe', 'a256e9af4', '509d9d60b', '3600c6e91',
        '5859a8a06', 'a02ae6a63', '1.0', 'Expert', 'Boiling Hot', 'b',
        'P', 'hT', '6.0', '2.0'],
       ['0.0', '0.0', '0.0', 'F', 'N', 'Blue', 'Trapezoid', 'Hamster',
        'Costa Rica', 'Piano', 'eaf88078e', '16e480d8e', 'd40ca0718',
        '153864851', '5533e4ecb', '1.0', 'Novice', 'Boiling Hot', 'h',
        'R', 'DI', 'nan', '5.0'],
       ['0.0', '0.0', '0.0', 'T', 'N', 'Blue', 'Circle', 'Hamster',
        'China', 'Piano', 'e7eaa3efc', '6f800b9af', '86ec768cd',
        '9fa5832d8', 'd7b264c91', '1.0', 'Master', 'Hot', 'g', 'P',
        'NaN', '3.0', '9.0']], dtype='<U22')

In [8]:
#import gc
#
#columns = df.columns
#unique_values_dict = {}
#for col in columns:
#  unique_val_objs = df.select(col).distinct().collect()
#  unique_val_list = [row[col] for row in unique_val_objs]
#  unique_values_dict[col] = unique_val_list
#  del unique_val_list
#  gc.collect()
#
#from math import e
#import random
#import numpy as np
#
#K = 3 #number of cluseters
#
##count_row
#n_data = df.count()
#
#for (i, col) in enumerate(columns):
#  unique_values = unique_values_dict[col]
#  ramdom_vals = random.choices(unique_values, k=K)
#  if i == 0:
#    centroid = np.array(ramdom_vals).reshape(-1, 1).astype('str')
#  else:
#    ramdom_vals = np.array(ramdom_vals).reshape(-1, 1).astype('str')
#    centroid = np.hstack((centroid, ramdom_vals))
#
#centroid = sc.broadcast(centroid)
#centroid.value

In [9]:
rdd = rdd.map(lambda row: to_numpy(row))

In [10]:
def hamming_distance(x1, x2):
  return np.count_nonzero(x1!=x2)

def get_closest_cluster_and_count(x, centroid):
  min_hamming_distance = np.inf
  closest_cluster = 0
  for i, mode in enumerate(centroid):
     distance = hamming_distance(x, mode)
     if distance < min_hamming_distance:
        min_hamming_distance = distance
        closest_cluster = i

  P = len(x)

  count_elem_hash = {}
  for i in range(P):
    count_elem_hash[i] = {x[i] : 1}

  return (closest_cluster, count_elem_hash)

def merge_count_elem_hash(count_elem_hash_A, count_elem_hash_B):
  for idx in count_elem_hash_A.keys():
    for key in count_elem_hash_B[idx].keys():
      if key in count_elem_hash_A[idx]:
        count_elem_hash_A[idx][key] += count_elem_hash_B[idx][key]
      else:
        count_elem_hash_A[idx][key] = 1
  return count_elem_hash_A

def reduce_helper_count_elem(store_hash_map, x):
  for i in range(len(x)):
    if x[i] in store_hash_map[i]:
      store_hash_map[i][x[i]] += 1
    else:
      store_hash_map[i][x[i]] = 1
  return store_hash_map

def get_centroid(count_elem_hash):
  P = len(count_elem_hash)
  centroid = np.full((P,), "")
  for idx, count_hash in count_elem_hash.items():
    mode = get_mode(count_hash)
    centroid[idx] = mode
  return centroid

def get_mode(count_hash):
  mode = ""
  highest_count = 0
  for value, count in count_hash.items():
    if count > highest_count:
      mode = value
      highest_count = count
  return mode

In [11]:
stop_distance = P//10

N_iters = 10

for iter in range(N_iters):

  clustered_and_hash_count_rdd = rdd.map(lambda x: get_closest_cluster_and_count(x, centroid.value)) #-> (k, v) = (cluster_i, count_hash)
  counted_elem_rdd = clustered_and_hash_count_rdd.reduceByKey(lambda x, y: merge_count_elem_hash(x, y))
  cluster_and_new_centroid_rdd =  counted_elem_rdd.map(lambda x: (x[0], get_centroid(x[1])))
  centroid_hash_form = cluster_and_new_centroid_rdd.collect()

  for idx in range(K):
    if idx == 0:
      new_centroid = np.array([centroid_hash_form[idx][1]])
    else:
      mode = np.array([centroid_hash_form[idx][1]])
      new_centroid = np.concatenate([new_centroid, mode])

  old_centroid = centroid.value.copy()
  centroid = sc.broadcast(new_centroid)

  distance = hamming_distance(old_centroid, new_centroid)

  print('iteration : ', {iter+1}, " hamming distance between new and previous centroid:  ", distance)

  if distance <= stop_distance:
    break

iteration :  {1}  hamming distance between new and previous centroid:   59
iteration :  {2}  hamming distance between new and previous centroid:   7
iteration :  {3}  hamming distance between new and previous centroid:   0
